注:相对原版本，未完成关于限制冲突分析检查

# alpha

In [1]:
import pandas as pd

from zipline import optimize as opt

from zipline.research import prices

In [2]:
# 尽量选择同时包含正负收益的股票列表
ps = prices(['000001','000002','000333','600645','600771'],'2018-5-19','2018-5-29')

[2018-06-02 06:19:41.945549] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-29 01:31:00+00:00
最后收盘时间: 2018-05-29 07:00:00+00:00


In [3]:
alphas = ps.pct_change(6).iloc[-1]

In [ ]:
alphas.index.difference

In [34]:
import numpy as np

In [31]:
class NotFoundAsset(Exception):
    """找不到对应资产异常"""
    pass

## 修改数据以利更好观察

In [4]:
alphas = pd.Series([-0.03,-0.10,-0.,0.03,0.11],index=alphas.index)

In [16]:
alphas

平安银行(000001)   -0.03
万 科Ａ(000002)   -0.10
美的集团(000333)   -0.00
中源协和(600645)    0.03
广誉远(600771)     0.11
dtype: float64

In [6]:
_long, _short = alphas.index[4], alphas.index[1]

# 优化

## 期初投资组合

In [7]:
current_portfolio = pd.Series([-0.2,-0.1,0.0001,0.1,0.2], index=alphas.index)

In [8]:
current_portfolio

平安银行(000001)   -0.2000
万 科Ａ(000002)   -0.1000
美的集团(000333)    0.0001
中源协和(600645)    0.1000
广誉远(600771)     0.2000
dtype: float64

## 最大alpha

In [9]:
objective = opt.MaximizeAlpha(alphas)

In [10]:
constraints = [
    #opt.MaxGrossExposure(3.0),
    #opt.NetExposure(-0.2,0.2),
    #opt.DollarNeutral(),
    #opt.Pair(_long, _short, 2),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
    #opt.Frozen(current_portfolio.index[2]),
    #opt.ReduceOnly(current_portfolio.index[:3]),
    #opt.LongOnly(current_portfolio.index[:2]),
    #opt.NotLessThan(0.1),
    opt.NotExceed(0.5),
    #opt.FixedWeight(current_portfolio.index[2],0.3),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
]

In [11]:
result = opt.run_optimization(objective, constraints, current_portfolio)

In [12]:
result.prob.status

'optimal'

In [13]:
# opt.MaxGrossExposure(3.0)与opt.NotExceed(0.6)组合时，出现没有意义的权重?
# 估计是收益率为0，计算精度问题?
# 如果分配完权重，则不受影响

In [14]:
result.new_weights

,long,short,total
平安银行(000001),0.00,-0.5000,0.5000
万 科Ａ(000002),0.00,-0.5000,0.5000
美的集团(000333),0.25,-0.1409,0.3909
中源协和(600645),0.50,-0.0000,0.5000
广誉远(600771),0.50,-0.0000,0.5000


# 单变量求解

In [15]:
import cvxpy as cvx

In [17]:
w = cvx.Variable(5)

In [18]:
obj = cvx.Maximize(alphas.values.T * w)

In [19]:
cons = [w >= -0.5, w <= 0.5]

In [20]:
prob = cvx.Problem(obj, cons)

In [21]:
prob.solve()

0.13499999999999998

In [22]:
w.value

array([-0.5, -0.5, -0. ,  0.5,  0.5])